<a href="https://colab.research.google.com/github/gagyeomkim/deepfake-defense-from-scratch/blob/seungeunlee/DDFSWEEK03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝을 이용한 자연어 처리

1. 텍스트의 토큰화

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
from numpy import array

# 텍스트 전처리 중 단어 단위로 문장을 분리하는 함수 -> keras
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 전처리할 텍스트 데이터
text = '해보지 않으면 해낼 수 없다'

# text_to_word_sequence() :
# 소문자 변환, 문장 부호 제거, 공백 기준으로 단어 토큰화
result = text_to_word_sequence(text)

# 원문 출력
print("\n원문:\n", text)
# 토큰화 된 결과 출력
print("\n토큰화:\n", result)


원문:
 해보지 않으면 해낼 수 없다

토큰화:
 ['해보지', '않으면', '해낼', '수', '없다']


In [5]:
# 단어 빈도수 세기

# 전처리하려는 세 개의 문장을 정합니다.
# 여기서 docs는 텍스트 데이터(코퍼스)의 역할을 합니다.
docs = [
    '먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
    '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
    '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.',
]

# Tokenizer 객체 생성
# - 이 객체는 단어들을 숫자로 변환하거나
# - 단어 빈도수, 문장 등장 수 등을 계산하는 기능을 가집니다.
token = Tokenizer()

# fit_on_texts():
# - docs 안의 문장들을 읽어서 단어별로 토큰화
# - 각 단어의 빈도수, 문장 등장 횟수, 인덱스 등을 계산
token.fit_on_texts(docs)

# token.word_counts
# - OrderedDict 형태로 저장됨
# - 각 단어가 전체 데이터에서 몇 번 등장했는지 저장
print("\n단어 카운트:\n", token.word_counts)

# token.document_count
# - 전체 문서(docs)의 개수
print("\n문장 카운트: ", token.document_count)

# token.word_docs
# - 각 단어가 몇 개의 문장에 포함되어 있는가:
# - 예: {'텍스트의': 2} → '텍스트의'가 두 문장에 등장했다는 뜻
print("\n각 단어가 몇 개의 문장에 포함되어 있는가:\n", token.word_docs)

# token.word_index
# - 단어별로 부여된 정수 인덱스
# - 빈도수가 높은 단어일수록 작은 숫자로 부여됨 (1번이 가장 높은 빈도)
print("\n각 단어에 매겨진 인덱스 값:\n", token.word_index)



단어 카운트:
 OrderedDict({'먼저': 1, '텍스트의': 2, '각': 1, '단어를': 1, '나누어': 1, '토큰화합니다': 1, '단어로': 1, '토큰화해야': 1, '딥러닝에서': 2, '인식됩니다': 1, '토큰화한': 1, '결과는': 1, '사용할': 1, '수': 1, '있습니다': 1})

문장 카운트:  3

각 단어가 몇 개의 문장에 포함되어 있는가:
 defaultdict(<class 'int'>, {'토큰화합니다': 1, '단어를': 1, '먼저': 1, '텍스트의': 2, '나누어': 1, '각': 1, '단어로': 1, '인식됩니다': 1, '토큰화해야': 1, '딥러닝에서': 2, '있습니다': 1, '사용할': 1, '결과는': 1, '수': 1, '토큰화한': 1})

각 단어에 매겨진 인덱스 값:
 {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화한': 11, '결과는': 12, '사용할': 13, '수': 14, '있습니다': 15}


2. 단어의 원-핫 인코딩

In [6]:
# 분석할 텍스트(문장)
text = "오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"

# Tokenizer 객체 생성
# - 단어를 토큰화하고, 각 단어에 고유한 정수 인덱스를 부여함
token = Tokenizer()

# fit_on_texts()
# Tokenizer가 텍스트를 읽고
# 단어(토큰) 빈도를 계산한 뒤
# 단어별 정수 인덱스를 자동으로 생성함
token.fit_on_texts([text])

# word_index
# 빈도수가 높은 단어일수록 더 낮은 인덱스(1부터 시작)를 부여함
# 이 문장은 모든 단어가 1번씩 나오므로 등장 순서대로 인덱스가 매겨짐
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [8]:
x=token.texts_to_sequences([text])

# 변환된 정수 sequence 출력
print(x)

[[1, 2, 3, 4, 5, 6]]


In [9]:
# +1을 하는 이유:
# word_index는 1부터 시작하기 때문에,
# 인덱스 0 자리도 포함시키기 위해 전체 크기를 하나 더 늘려야 함
word_size = len(token.word_index) + 1

# x -> 정수 시퀀스 형태: [[1, 2, 3, 4, 5, 6]]
# 이를 원-핫 인코딩(one-hot encoding)으로 변환
# num_classes 매개변수에 전체 단어 수(word_size)를 넣어주기
x = to_categorical(x, num_classes=word_size)

# 인코딩 결과 출력
print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


4. 텍스트를 읽고 긍정, 부정 예측하기

In [10]:
# 텍스트 리뷰 자료를 지정
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스를 지정
classes = array([1,1,1,1,1,0,0,0,0,0])

# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)


{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [12]:
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n",  x)


리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [13]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞추어 줌.
padded_x = pad_sequences(x, 4)
print("\n패딩 결과:\n", padded_x)


패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [14]:
# 임베딩에 입력될 단어의 수를 지정합니다.
# Tokenizer로 만들어진 단어 사전(word_index)은 1부터 시작함
# 0번 인덱스를 포함시키기 위해 +1
word_size = len(token.word_index) + 1

# 딥러닝 모델 선언 (Sequential 방식)
model = Sequential()

# Embedding 층
# word_size : 전체 단어 수 (정수 인덱스 범위)
# 8 : 각 단어를 8차원 벡터로 임베딩 (embedding_dim)
# input_length=4 :
# 모델이 입력으로 받는 문장의 길이(패딩된 길이)
# 예: [0, 0, 1, 2] 처럼 길이가 4인 정수 시퀀스
model.add(Embedding(word_size, 8, input_length=4))

# Flatten 층
# Embedding의 출력은 2D 텐서(문장길이 × embedding_dim)
# Flatten은 이를 1D로 펼쳐서 Dense층에 넣기 편하도록 만들어줌
model.add(Flatten())

# Dense 층
# 출력 1개 (이진 분류용)
# sigmoid 활성화 함수 사용 → 0~1 사이 결과
model.add(Dense(1, activation='sigmoid'))

# 모델 구조 요약 출력
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
# 모델 학습을 위한 설정
# optimizer='adam'  : 학습률 자동 조정 기능이 있는 Adam 옵티마이저 사용
# loss='binary_crossentropy' : 이진 분류 문제에서 사용하는 손실 함수
# metrics=['accuracy']       : 정확도를 평가 지표로 사용
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
# padded_x  : 학습 데이터 (정수 인덱스 또는 임베딩 입력 형태)
# classes   : 각 입력 문장에 대한 정답(레이블)
# epochs=20 : 20번 반복 학습
model.fit(padded_x, classes, epochs=20)

# 모델 평가
# evaluate(입력, 정답)은 [loss, accuracy] 형태로 값을 반환
# [1] 은 accuracy 값을 의미함
print("\nAccuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6000 - loss: 0.6874
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7000 - loss: 0.6853
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7000 - loss: 0.6832
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7000 - loss: 0.6811
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.7000 - loss: 0.6790
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8000 - loss: 0.6768
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8000 - loss: 0.6747
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9000 - loss: 0.6726
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9000 - loss: 0.6705
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9000 - loss: 0.6684
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9000 - loss: 0.6662
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9000 - loss: 0.6641
Epo